In [1]:
import numpy as np
import pandas as pd

# RANDOM TIME DATA
date_rng = pd.date_range(start='1/1/2021', end='1/2/2021', freq='s')
df = pd.DataFrame(date_rng, columns=['timestamp'])

np.random.seed(42)
df['value'] = np.random.randint(0, 100, size=(len(date_rng)))
df = df.sample(frac=0.5, random_state=42).sort_values(by=['timestamp'])
df.to_csv('data.csv', index=False)

,timestamp,value
0,2021-01-01 00:00:00,51
4,2021-01-01 00:00:04,60
6,2021-01-01 00:00:06,82
7,2021-01-01 00:00:07,86
11,2021-01-01 00:00:11,99


In [ ]:
import csv
rdr = csv.reader(open(data.csv))
next(rdr)

In [ ]:
import argparse
import json
import sys
import time
import socket
from kafka import KafkaConsumer, errors


def msg_process(msg):
    # Print the current time and the message.
    time_start = time.strftime("%Y-%m-%d %H:%M:%S")
    val = msg.value()
    dval = json.loads(val)
    print(time_start, dval)


parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument('topic', type=str, help='Name of the Kafka topic to stream.')

args = parser.parse_args()

conf = {'bootstrap.servers': 'localhost:9092',
            'default.topic.config': {'auto.offset.reset': 'smallest'},
            'group.id': socket.gethostname()}

consumer = KafkaConsumer('dummy_topic',bootstrap_servers='localhost:9092',
                        value_deserializer=lambda m:json.loads(m.decode('ascii')))

try:
    while True:
        msg = consumer.poll(1)
        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                    (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error().code() == KafkaError.UNKNOWN_TOPIC_OR_PART:
                sys.stderr.write('Topic unknown, creating %s topic\n' %
                                    (args.topic))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            msg_process(msg)

except KeyboardInterrupt:
    pass

finally:
    # Close down consumer to commit final offsets.
    consumer.close()